In [1]:
from random import randint, choices
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import re
from tqdm import tqdm, trange

In [2]:
K = 4
alpha = .1
beta = .1

In [3]:
from os import listdir

def fileids(path, ext='txt'):
    path = path if path[-1] == '/' else path+'/'
    fileList = list()
    for fileName in listdir(path):
        if fileName.endswith(ext):
            fileList.append(path+fileName)
    return fileList


In [5]:
ma = Okt()
path = '../../../SWacademy/Data collecting/news'

corpus = list()

for filename in fileids(path):
    with open(filename, encoding='utf-8') as fp:
        cate = re.search((r'(\d{3})'), filename).groups(1)[0]
        cleanText = re.sub(r'\s{2,}', ' ', fp.read().strip())
        corpus.append(ma.nouns(cleanText))
        # corpus.append(ma.nouns(sent_tokenize(cleanText)))

# word

In [6]:
corpus[0]

['고민정',
 '민주당',
 '의원',
 '고민정',
 '민주당',
 '의원',
 '윤석열',
 '대통령',
 '향',
 '문재인',
 '학생',
 '열등감',
 '학생',
 '보이지',
 '고',
 '지적',
 '고',
 '의원',
 '자신',
 '페이스북',
 '해외',
 '현직',
 '국가원수',
 '임자',
 '폄훼',
 '비난',
 '경우',
 '처음',
 '자기',
 '얼굴',
 '침',
 '모습',
 '보이지',
 '며',
 '모든',
 '학생',
 '반',
 '리더십',
 '반장',
 '면서',
 '우리',
 '반',
 '옆',
 '반',
 '학생',
 '신망',
 '반장',
 '열등감',
 '더',
 '귀',
 '더',
 '만남',
 '고',
 '비판',
 '현재',
 '감사원',
 '전',
 '정부',
 '대한',
 '먼지털이',
 '감사',
 '진행',
 '국민',
 '힘',
 '사사건건',
 '민주당',
 '의원',
 '전',
 '정부',
 '인사',
 '고발',
 '면서',
 '얼마',
 '전',
 '국민',
 '힘',
 '기무사',
 '계엄령',
 '문건',
 '관련',
 '전',
 '정부',
 '인사',
 '고발',
 '날',
 '국민',
 '힘',
 '송영',
 '무',
 '이석구',
 '임태훈',
 '등',
 '관련',
 '인사',
 '직권남용',
 '등',
 '혐의',
 '고발',
 '고',
 '최근',
 '사건',
 '언급',
 '또',
 '감사원',
 '국민',
 '힘',
 '모두',
 '전',
 '정부',
 '대한',
 '열등감',
 '똘똘',
 '것',
 '생각',
 '면서',
 '최근',
 '민주당',
 '감사원',
 '정치',
 '개입',
 '방지법',
 '민주당',
 '삼권분립',
 '입법부',
 '역할',
 '고',
 '강조',
 '여기',
 '국민',
 '힘',
 '정당',
 '본연',
 '임무',
 '모두',
 '검찰',
 '바치',
 '윤',
 '대통령',
 '그림자',
 '역할',
 '게',
 '정당',


In [5]:
theta = dict() # 문서-토픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
phi = dict() # 단어픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
z = dict() # 문서-단어 토픽 할당 정보 => m번째 문서, l번째 단어 => {m:{l:k}}

V = list()
for m, d in enumerate(corpus):
    for l, w in enumerate(d):
        V.append(w.lower())
        if m not in z:
            z[m] = dict()
        if l not in z[m]:
            z[m][l] = 0
        z[m][l] = randint(1,K) # 토픽 할당

V = set(V)
i2w = {i: v for i, v in enumerate(V)}
w2i = {v: i for i, v in enumerate(V)}

for m, L in z.items():
    if m not in theta:
        theta[m] = dict()
    for l, t in L.items():
        if t not in theta[m]:
            theta[m][t] = 0
        theta[m][t] += 1

        v = w2i[corpus[m][l].lower()]

        if v not in phi:
            phi[v] = dict()
        if t not in phi[v]:
            phi[v][t] = 0
        phi[v][t] += 1

In [ ]:
iteration = 100

for _ in trange(iteration):
    for m, d in enumerate(corpus):
        for l, w in enumerate(d):
            v = w2i[w.lower()]

            k = z[m][l] # gibb's 대상 삭제
            theta[m][k] -= 1
            phi[v][k] -= 1

            topic = list()
            for t in range(K):
                A = (phi[v].get(t+1,0) + beta) / sum([T.get(t+1,0)+beta for v, T in phi.items()])
                B = theta[m].get(t+1,0) + alpha
                topic.append(A*B)

            k = choices(range(K), weights=topic, k=1)[0]+1

            z[m][l] = k
            theta[m][k] = (theta[m][k] if k in theta[m] else 0) + 1
            phi[v][k] = (phi[v][k] if k in phi[v] else 0) + 1
    print('{}%'.format(_))

KeyboardInterrupt: 

In [ ]:
for l, T in phi.items():
    if T[1] > 0:
        print(i2w[l], T[1])

In [ ]:
phi